In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.integrate import simps
from my_functions import *

In [ ]:
N_spec = 1000
binning = 10000

#Parameters' range
f_in = [-19., -16. ]
e_in = [10  , 100  ]
w_in = [2500, 10000]
z_in = [2.8 , 3.2  ]

In [ ]:
z_Arr = np.random.rand(N_spec)*(z_in[1] - z_in[0]) + z_in[0]
e_Arr = np.random.rand(N_spec)*(e_in[1] - e_in[0]) + e_in[0]
w_Arr = np.linspace(w_in[0], w_in[1], binning)
f_Arr = 10**(np.random.rand(N_spec)*(f_in[1] - f_in[0]) + f_in[0])

In [ ]:
# Continuum
cont = f_Arr*np.ones((binning, N_spec))
cont = cont.transpose()

In [ ]:
# Line
w_Lya = 1215.67

lines = np.zeros((N_spec, binning))
F_line = e_Arr*(1+z_Arr)*f_Arr - f_Arr

for i in range(N_spec):
    lines[i, :] = F_line[i]*norm.pdf(w_Arr, (z_Arr[i]+1)*w_Lya, 5)

In [ ]:
# Continuum + line
spec = cont + lines

In [ ]:
# Plot some spectra
fig, axs = plt.subplots(4, 2, figsize=(15,7))
k = 0
for i in range(4):
    for j in range(2):
        axs[i,j].plot(w_Arr, spec[k, :])
        k += 1

In [ ]:
# T curve of top-hat filter
fl_nb_w    = 4902. # As in J0490
fl_nb_fwhm = 154.  # As in J0490
fl_nb_z = fl_nb_w/w_Lya - 1

fl_bb_w    = 4750.  # As in gSDSS
fl_bb_fwhm = 1250.  # As in gSDSS

tcurve_nb = np.zeros(binning)
tcurve_nb[np.where(np.abs(w_Arr - fl_nb_w) < fl_nb_fwhm/2)] += 1

tcurve_bb = np.zeros(binning)
tcurve_bb[np.where(np.abs(w_Arr - fl_bb_w) < fl_bb_fwhm/2)] += 1

fig, axs = plt.subplots(figsize = (15, 7))
axs.plot(w_Arr, tcurve_nb)
axs.plot(w_Arr, tcurve_bb)
plt.show()

In [ ]:
# Photometry
pm_nb = []
pm_bb = []
for i in range(N_spec):
    pm_aux = (simps(spec[i, :]*tcurve_nb*w_Arr, w_Arr)/simps(tcurve_nb*w_Arr, w_Arr),
              simps(spec[i, :]*tcurve_bb*w_Arr, w_Arr)/simps(tcurve_bb*w_Arr, w_Arr))
    pm_nb.append(pm_aux[0])
    pm_bb.append(pm_aux[1])

In [ ]:
# Plot
nb = flux_to_mag(np.array(pm_nb), fl_nb_w)
bb = flux_to_mag(np.array(pm_bb), fl_bb_w)

plt.figure(figsize=(10,7))
plt.scatter(nb, bb-nb)
plt.xlabel('NB', size=15)
plt.ylabel('BB - NB', size=15)
plt.show()

In [ ]:
# EW test
def ew_theo(bbnb_x):
    return (10**(bbnb_x/2.5) - 1) * fl_nb_fwhm/(1+fl_nb_z)

bbnb_x = np.linspace(0, 1.5, 100)

plt.figure(figsize=(10,7))
plt.scatter(bb-nb, e_Arr)
plt.plot(bbnb_x, ew_theo(bbnb_x), c='red')
plt.xlim((0,1.4))
plt.xlabel('BB - NB', size=15)
plt.ylabel('EW$_0$' , size=15)
plt.show()